In [21]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("西瓜数据集3.0.txt", index_col="编号")
df.head()

,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
编号,,,,,,,,,
1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是


In [3]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [4]:
features = list(df.columns)
label = '好瓜'
features.remove(label)
label, features

('好瓜', ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '密度', '含糖率'])

In [5]:
# 标签二值化
label_bi = LabelBinarizer()
df[label] = label_bi.fit_transform(df[label])
# label_bi.classes_  # array(['否', '是'], dtype='<U1')

In [6]:
# 加入测试集
test = ['青绿', '蜷缩', '浊响', '清晰'
       , '凹陷', '硬滑', 0.697, 0.460, -1]
df.loc[len(df)+1] = test

In [7]:
# 类别编码化
for ft in df[features].select_dtypes(['object']):
    print()
    df[ft] = LabelEncoder().fit_transform(df[ft])
    print()

In [8]:
df.head()

,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
编号,,,,,,,,,
1,2,2,1,1,0,0,0.697,0.460,1
2,0,2,0,1,0,0,0.774,0.376,1
3,0,2,1,1,0,0,0.634,0.264,1
4,2,2,0,1,0,0,0.608,0.318,1
5,1,2,1,1,0,0,0.556,0.215,1


In [50]:
# 训练集
train_x = df[df[label]!=-1][features]
train_y = df[df[label]!=-1][[label]]
# 评估集
eval_x = df[df[label]==-1][features]
eval_y = df[df[label]==-1][[label]]

In [82]:
# 计算P(一列类别取值)
def col_prob(col_value, col):
    """
    col_value: int or float
    y 传入ndarray
    """
    if col.dtype in ['float32', 'float64']:
        # 浮点型特征另外处理
        mean = col.mean()
        std = col.std()
        prob = np.exp(-(col_value-mean)**2/ (2*std**2)) / (np.sqrt(np.pi*2) * std)
    # 统计等于y_value 的值的个数
    else:
        num = np.where(col==col_value, 1, 0).sum()
        prob = num / len(col)
    return prob

In [94]:
col_prob(0.697,train_x[train_y[label]==1]["密度"])

1.9590115494650384

In [95]:
col_prob(0.697,train_x[train_y[label]==0]["密度"])

1.2033038984540718

In [97]:
col_prob(0.460,train_x[train_y[label]==1]["含糖率"])

0.7880520952044108

In [98]:
col_prob(0.460,train_x[train_y[label]==0]["含糖率"])

0.06622115248436898

# 展示西瓜书中间结果

In [109]:
# 西瓜书p84
df = pd.read_csv("西瓜数据集3.0.txt", index_col="编号")
df

,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
编号,,,,,,,,,
1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否


In [102]:
# 计算P(一列类别取值)
def col_prob(col_value, col):
    """
    col_value: str, int, float
    col pd.DataFrame 的一列
    """
    if col.dtype in ['float32', 'float64']:
        # 浮点型特征另外处理
        mean = col.mean()
        std = col.std()
        # 高斯分布
        prob = np.exp(-(col_value-mean)**2/ (2*std**2)) / (np.sqrt(np.pi*2) * std)
    # 统计等于y_value 的值的个数
    else:
        num = np.where(col==col_value, 1, 0).sum()
        prob = num / len(col)
    return prob

In [140]:
p_好瓜_是 = col_prob('是',df['好瓜'])
p_好瓜_否 = col_prob('否',df['好瓜'])

p_青绿_是 = col_prob('青绿',df[df['好瓜']=='是']["色泽"])
p_青绿_否 = col_prob('青绿',df[df['好瓜']=='否']["色泽"])

p_蜷缩_是 = col_prob('蜷缩',df[df['好瓜']=='是']["根蒂"])
p_蜷缩_否 = col_prob('蜷缩',df[df['好瓜']=='否']["根蒂"])

p_浊响_是 = col_prob('浊响',df[df['好瓜']=='是']["敲声"])
p_浊响_否 = col_prob('浊响',df[df['好瓜']=='否']["敲声"])

p_清晰_是 = col_prob('清晰',df[df['好瓜']=='是']["纹理"])
p_清晰_否 = col_prob('清晰',df[df['好瓜']=='否']["纹理"])

p_凹陷_是 = col_prob('凹陷',df[df['好瓜']=='是']["脐部"])
p_凹陷_否 = col_prob('凹陷',df[df['好瓜']=='否']["脐部"])

p_硬滑_是 = col_prob('硬滑',df[df['好瓜']=='是']["触感"])
p_硬滑_否 = col_prob('硬滑',df[df['好瓜']=='否']["触感"])

# 数值型特征
p_密度_是 = col_prob(0.697,df[df['好瓜']=='是']["密度"])
p_密度_否 = col_prob(0.697,df[df['好瓜']=='否']["密度"])

p_含糖率_是 = col_prob(0.460,df[df['好瓜']=='是']["含糖率"])
p_含糖率_否 = col_prob(0.460,df[df['好瓜']=='否']["含糖率"])

In [141]:
print('西瓜书p152 - 153\n')
print('p_好瓜_是', round(p_好瓜_是,3))
print('p_好瓜_否', round(p_好瓜_否,3))

print('p_青绿_是', round(p_青绿_是,3))
print('p_青绿_否', round(p_青绿_否,3))

print('p_蜷缩_是', round(p_蜷缩_是,3), '***')
print('p_蜷缩_否', round(p_蜷缩_否,3))

print('p_浊响_是', round(p_浊响_是,3))
print('p_浊响_否', round(p_浊响_否,3))

print('p_清晰_是', round(p_清晰_是,3))
print('p_清晰_否', round(p_清晰_否,3))

print('p_凹陷_是', round(p_凹陷_是,3), '***')
print('p_凹陷_否', round(p_凹陷_否,3))

print('p_硬滑_是', round(p_硬滑_是,3))
print('p_硬滑_否', round(p_硬滑_否,3))

print('p_密度_是', round(p_密度_是,3))
print('p_密度_否', round(p_密度_否,3))

print('p_含糖率_是', round(p_含糖率_是,3))
print('p_含糖率_否', round(p_含糖率_否,3))

西瓜书p152 - 153

p_好瓜_是 0.471
p_好瓜_否 0.529
p_青绿_是 0.375
p_青绿_否 0.333
p_蜷缩_是 0.625 ***
p_蜷缩_否 0.333
p_浊响_是 0.75
p_浊响_否 0.444
p_清晰_是 0.875
p_清晰_否 0.222
p_凹陷_是 0.625 ***
p_凹陷_否 0.222
p_硬滑_是 0.75
p_硬滑_否 0.667
p_密度_是 1.959
p_密度_否 1.203
p_含糖率_是 0.788
p_含糖率_否 0.066


对比一下可以发现西瓜书有几处错误
![朴素贝叶斯计算结果1.png](朴素贝叶斯计算结果1.png)

In [143]:
p_是好瓜 = p_好瓜_是 * \
            p_青绿_是 * p_蜷缩_是 * p_浊响_是 * p_清晰_是 * \
            p_凹陷_是 * p_硬滑_是 * p_密度_是 * p_含糖率_是
p_否好瓜 = p_好瓜_否 * \
            p_青绿_否 * p_蜷缩_否 * p_浊响_否 * p_清晰_否 * \
            p_凹陷_否 * p_硬滑_否 * p_密度_否 * p_含糖率_否

In [146]:
p_是好瓜, p_否好瓜

(0.05237871892595563, 6.85842403030207e-05)